In [1]:
import numpy
import pandas

import gensim
from gensim.corpora import Dictionary
from gensim.models import TfidfModel

from pythainlp.tokenize import word_tokenize
import pythainlp

from sklearn.cluster import KMeans

import re
import math

### Load Corpus

In [2]:
file_directory = '../data/facebook/2/ผู้บริโภค - TescoLotus.txt'
corpus = []
labels = []
    
f = open(file_directory, 'r')
data = f.read().splitlines()
f.close()
    
start = False
num = -2

for line in data:
    if line.startswith('comment'):
        num += 1
        if num < 0:
            continue

        comment = ''.join(x for x in line.split(':')[2:])
        corpus.insert(num, comment)
        labels.insert(num, line.split(':')[1])
        start = True

    elif start:
        corpus[num] += line

In [3]:
len_corpus = len(corpus)
print('Total documents', len_corpus)

clusters = list(set(labels))
print(len(clusters), 'clusters')

Total documents 273
1 clusters


### Preprocess Corpus

#### Tokenize Corpus

In [4]:
def clean(doc):
    doc.strip()
    while True:
        new_doc = re.sub('[^" "\u0E00-\u0E7F]+', '', doc)
        if doc == new_doc:
            break
        else:
            doc = new_doc
    return doc

tokenized_corpus = []
for doc in corpus:
    cleaned_doc = clean(doc)
    tokenized_corpus.append(word_tokenize(cleaned_doc, engine='deepcut'))

Using TensorFlow backend.


#### Remove Words

In [5]:
dictionary = Dictionary(tokenized_corpus)
print('origin:', len(dictionary), 'words')

appear_rate = 1
dictionary.filter_extremes(no_below=2, no_above=0.7, keep_n=len(dictionary))
print('filter frequent words:', len(dictionary), 'words')

letter_words = []
for word in dictionary.keys():
    if len(dictionary[word]) <= 1:
        letter_words.append(word)
dictionary.filter_tokens(bad_ids=letter_words)
print('filter letter words:', len(dictionary), 'words')

stopwords = pythainlp.corpus.stopwords.words('thai')
stopwords.append('นี้')
dictionary.add_documents([stopwords])
stopwords = [dictionary.token2id[word] for word in stopwords]
dictionary.filter_tokens(bad_ids=stopwords)
print('filter stop words:', len(dictionary), 'words')

origin: 1327 words
filter frequent words: 552 words
filter letter words: 551 words
filter stop words: 360 words


In [6]:
# bow_corpus = [dictionary.doc2bow(doc) for doc in tokenized_corpus]
idx_corpus = [dictionary.doc2idx(doc) for doc in tokenized_corpus]

temp_corpus = []
for doc in idx_corpus:
    temp_corpus.append([dictionary[id] for id in doc if id >= 0])
    print(temp_corpus[-1])
idx_corpus = temp_corpus

['พนักงาน', 'ทำ', 'หน้า', 'ตูด', 'คน', 'เหมือน', 'บังคับ', 'ทำ', 'งาน', 'งั้น', 'เครียด']
['ถุง', 'งง', 'ถุง', 'กก', 'เจอ', 'น้ำยา', 'ซัก', 'ผ้า', 'น้ำ', 'ยา', 'ผ้า', 'นุ่ม']
['ถุง', 'แถม', 'ลด', 'ลอง', 'ดู', 'ถุง', 'หนา']
['อย่า', 'บังคับ', 'น้อง', 'เชียร์', 'อย่า', 'งี่', 'เง่า', 'กี่', 'ปี']
['เอ็กเพรส', 'อบรม', 'พนักงาน', 'ป่าว', 'อารม', 'ถาม', 'ราคา', 'ทำ', 'หน้า', 'พอใจ', 'ติด', 'ราคา', 'ถาม', 'หรอก', 'ราคา', 'แถม', 'ยิง', 'สมาชิก', 'ยิง', 'ลด', 'ทำ', 'หน้า', 'กด', 'ร้าน']
['โลตัส', 'โลตัส', 'เอกเพรส', 'แม่ง', 'เกลียด', 'พนักงาน']
['ทำ', 'เดือน', 'พนักงาน', 'คน', 'คน', 'คน', 'นึง', 'พัน', 'เช้า', 'เย็น']
['มารยาท', 'พนักงาน', 'เซเว่น', 'คุณภาพ', 'ดี', 'นึง', 'เหี้ย', 'คู่']
['ราคา', 'สินค้า', 'ตอน', 'โปรโมชั่น', 'หา', 'เจอ', 'เหมือน', 'พนักงาน', 'ซ่อน', 'ลูกค้า', 'วิ่ง', 'ไล่', 'หา', 'ไอ้', 'ซ่อน', 'ผ้า', 'หน้าตา', 'เวลา', 'ห้าง', 'นึก', 'เวลา']
['เลิก', 'ข้าว', 'ดี', 'อี', 'เครื่อง', 'น้ำ']
['เช็ค', 'หน้า', 'งอ', 'สวัสดี']
['เค้าเตอร์', 'ช่อง', 'จ่าย', 'ตัง', 'ช่อง', 'ทำ', 'หรอ'

In [7]:
unique_words = [dictionary[id] for id in range(len(dictionary))]

array = numpy.zeros((len_corpus, len(unique_words)), dtype=int)
for i, vector in enumerate(idx_corpus):
    for word in vector:
        array[i, dictionary.token2id[word]] += 1

word_vectors = pandas.DataFrame(array, columns=unique_words, dtype=int)
word_vectors.head()

360 words


,คน,งั้น,งาน,ตูด,ทำ,บังคับ,พนักงาน,หน้า,เครียด,เหมือน,...,ฉีด,ล้อ,สะอาด,ร้อน,แจก,เขียว,พลังงาน,แอร์ไม่,สลิป,นวมินทร์
0,1,1,1,1,2,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,2,0,1,2,0,0,...,0,0,0,0,0,0,0,0,0,0


### K-Mean

In [8]:
kmeans = KMeans(n_clusters=5).fit(word_vectors)
predicted_labels = kmeans.labels_

In [9]:
result = pandas.DataFrame()
result['comment'] = corpus
result['tokenized_comment'] = idx_corpus
result['label'] = labels
result['predicted_label'] = predicted_labels
result.head()

,comment,tokenized_comment,label,predicted_label
0,พนักงานทำหน้าเป็นตูดเกือบทุกคน ประมาณว่าเหมือน...,"[พนักงาน, ทำ, หน้า, ตูด, คน, เหมือน, บังคับ, ท...",,4
1,ถุงงง ถุงมึงบอบบางมากกก ยิ่งเจอน้ำยาซักผ้า น้...,"[ถุง, งง, ถุง, กก, เจอ, น้ำยา, ซัก, ผ้า, น้ำ, ...",,1
2,เราไม่เคยรับถุงเลย แถมได้แต้มกรีนพ้อยไว้ใช้เป็...,"[ถุง, แถม, ลด, ลอง, ดู, ถุง, หนา]",,1
3,อย่าบังคับน้องเข้าประชุมเชียร์ อย่าลงโทษโดยเหต...,"[อย่า, บังคับ, น้อง, เชียร์, อย่า, งี่, เง่า, ...",,1
4,เอ็กเพรสนี่มีการอบรมพนักงานบ้างป่าว ควบคุมอารม...,"[เอ็กเพรส, อบรม, พนักงาน, ป่าว, อารม, ถาม, ราค...",,4


In [10]:
label_count = numpy.unique(predicted_labels, return_counts=True) 
print(label_count)

for cluster in clusters:
    print('\t' + cluster, end='')
print('\tpercent')

for label in range(len(clusters)):
    print(str(label) + '  |', end='')
    
    num_max = 0
    for cluster in clusters:
        loc = result[(result['label'] == cluster) & (result['predicted_label'] == label)]
        if len(loc) > num_max:
            num_max = len(loc)
        print('\t' + str(len(loc)), end='')
    
    print('\t' + str(num_max / label_count[1][label]))

(array([0, 1, 2, 3, 4], dtype=int32), array([  6, 239,   4,   2,  22]))
		percent
0  |	6	1.0


In [11]:
for index, value in result[result['predicted_label'] == 4]['comment'].iteritems():
    print(index, '[', value, ']')

0 [ พนักงานทำหน้าเป็นตูดเกือบทุกคน ประมาณว่าเหมือนถูกบังคับให้มาทำงานยังงั้นแหละ เห็นแล้วพลอย เครียดไปด้วย ]
4 [ เอ็กเพรสนี่มีการอบรมพนักงานบ้างป่าว ควบคุมอารมไม่เป็น ถามราคาต้องทำหน้าไม่พอใจคือถ้าติดราคาก็ไม่ถามหรอกราคาไม่มี แถมให้ยิงบาโค้ดสมาชิก ยิงส่วนลดต้องทำหน้าแบบรำคาน กดเครื่องแรงๆประชดประชัน เข้าร้านอื่นก็ได้วะ ]
12 [ พนักงานเหมือนไม่อยากทำงาน หน้าหงิกหน้างอ แต่ก็มีบางคนที่น่ารัก พูดดีส่วนน้อย ]
16 [ พนักงานเอ็กเพรส บางสาขาหน้าบูดมาก  และทำงานช้ามากกกกกกถามอะไร ไม่ค่อยรู้เรื่อง แก้ไขสถานการณ์เฉพาะหน้าได้ไม่ดี ]
20 [ พนักงานโลตัสสัตหีบบริการดีฮะ  แต่อยากบอกว่าถ้าลูกค้าประจำที่คุณจำหน้าได้มาซื้อของไม่ต้องตะโกนบอกเพื่อนก็ได้ฮะว่า"เฮ้ยๆ คนนี้เค้าไม่ชอบให้ใส่ถุง"    แบบบอกแค่ว่า "ลูกค้าเข็นของไปใส่ท้ายรถเอง " ก็ได้  ผมอายฮะ ]
25 [ เราก็ไม่รู้ระบบการทำงานของเค้านะ แต่เวลาไปซื้อของชอบเอาของมาเติม ลากเกะกะกันไปหมดทั้งรถเข็นคนซื้อทั้งรถพนักงาน แล้วก็ติดขัดจะไปต่อก็ไม่ได้จะถอยก็ไม่ได้ แล้วไม่ใช่จะรีบเติมให้เสร็จนะ ยืนเล่นกันแหย่กัน เกะกะไปหมด โลตัสเอ็กซ์เพรส เวลาจ่ายด้วยบัตรเครดิตเครื่อง

In [12]:
for index, value in result[result['predicted_label'] == 4]['tokenized_comment'].iteritems():
    print(index, value)

0 ['พนักงาน', 'ทำ', 'หน้า', 'ตูด', 'คน', 'เหมือน', 'บังคับ', 'ทำ', 'งาน', 'งั้น', 'เครียด']
4 ['เอ็กเพรส', 'อบรม', 'พนักงาน', 'ป่าว', 'อารม', 'ถาม', 'ราคา', 'ทำ', 'หน้า', 'พอใจ', 'ติด', 'ราคา', 'ถาม', 'หรอก', 'ราคา', 'แถม', 'ยิง', 'สมาชิก', 'ยิง', 'ลด', 'ทำ', 'หน้า', 'กด', 'ร้าน']
12 ['พนักงาน', 'เหมือน', 'ทำ', 'งาน', 'หน้า', 'งอ', 'คน', 'รัก', 'ดี']
16 ['พนักงาน', 'เอ็กเพรส', 'สาขา', 'หน้า', 'บูด', 'ทำ', 'งาน', 'กกกกก', 'ถาม', 'รู้', 'เรื่อง', 'ดี']
20 ['พนักงาน', 'บริการ', 'ดี', 'ฮะ', 'ลูกค้า', 'ประจำ', 'หน้า', 'ซื้อ', 'ตะโกน', 'เพื่อน', 'ฮะ', 'เฮ้ย', 'คน', 'เค้า', 'ชอบ', 'ใส่', 'ถุง', 'ลูกค้า', 'เข็น', 'ใส่', 'รถ', 'ผม']
25 ['รู้', 'ระบบ', 'ทำ', 'งาน', 'เค้า', 'เวลา', 'ซื้อ', 'ชอบ', 'ลาก', 'เกะกะ', 'รถ', 'เข็น', 'คน', 'ซื้อ', 'รถ', 'พนักงาน', 'รีบ', 'เติม', 'ยืน', 'เล่น', 'เกะกะ', 'โลตัส', 'เอ็กซ์เพรส', 'เวลา', 'จ่าย', 'บัตร', 'เครื่อง', 'พนักงาน', 'แย่']
33 ['โลตัส', 'สาขา', 'จ้าง', 'พนักงาน', 'พนักงาน', 'แผนก', 'คน', 'หน้าที่', 'ตัว', 'แคชเชียร์', 'งาน', 'แผนก', 'ตัว', 'โดน', 'ด่า

### Dimension Reduction

In [13]:
average_doc_size = 0
for doc in idx_corpus:
    average_doc_size += len(doc)
average_doc_size /= len(idx_corpus)
average_doc_size = math.ceil(average_doc_size)
average_doc_size

word_freq = dictionary.dfs
filtered_corpus = []
for doc in idx_corpus:
    new_doc = [(word, word_freq[dictionary.token2id[word]]) for word in doc]
    new_doc.sort(reverse=True, key=lambda x: x[1])
    new_doc = new_doc[:average_doc_size]
    filtered_corpus.append([word for word, df in new_doc])

In [14]:
dictionary = Dictionary(filtered_corpus)

# dictionary.filter_extremes(no_below=2, no_above=1, keep_n=len(dictionary))
# print(len(dictionary))

unique_words = [dictionary[id] for id in range(len(dictionary))]

array = numpy.zeros((len_corpus, len(unique_words)), dtype=int)
for i, vector in enumerate(filtered_corpus):
    for word in vector:
        array[i, dictionary.token2id[word]] += 1

word_vectors = pandas.DataFrame(array, columns=unique_words, dtype=int)
word_vectors.head()

283 words


,คน,งาน,ตูด,ทำ,บังคับ,พนักงาน,หน้า,เหมือน,กก,งง,...,แอร์ไม่,สลิป,คู,ปอง,นวมินทร์,ซ่อม,ลำบาก,สิบ,ห้า,แจก
0,1,1,1,2,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,2,0,1,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
kmeans = KMeans(n_clusters=5).fit(word_vectors)
predicted_labels = kmeans.labels_

In [16]:
result = pandas.DataFrame()
result['comment'] = corpus
result['tokenized_comment'] = filtered_corpus
result['label'] = labels
result['predicted_label'] = predicted_labels
result.head()

,comment,tokenized_comment,label,predicted_label
0,พนักงานทำหน้าเป็นตูดเกือบทุกคน ประมาณว่าเหมือน...,"[พนักงาน, ทำ, หน้า, ทำ, คน, เหมือน, งาน, ตูด, ...",,0
1,ถุงงง ถุงมึงบอบบางมากกก ยิ่งเจอน้ำยาซักผ้า น้...,"[ถุง, ถุง, เจอ, น้ำ, งง, น้ำยา, ผ้า, ผ้า, กก]",,2
2,เราไม่เคยรับถุงเลย แถมได้แต้มกรีนพ้อยไว้ใช้เป็...,"[ถุง, ถุง, ลด, แถม, ดู, หนา, ลอง]",,2
3,อย่าบังคับน้องเข้าประชุมเชียร์ อย่าลงโทษโดยเหต...,"[อย่า, อย่า, กี่, ปี, บังคับ, น้อง, เชียร์, งี...",,4
4,เอ็กเพรสนี่มีการอบรมพนักงานบ้างป่าว ควบคุมอารม...,"[พนักงาน, ทำ, หน้า, ทำ, หน้า, ถาม, ถาม, ลด, ราคา]",,0


In [17]:
label_count = numpy.unique(predicted_labels, return_counts=True) 
print(label_count)

for cluster in clusters:
    print('\t' + cluster, end='')
print('\tpercent')

for label in range(len(clusters)):
    print(str(label) + '  |', end='')
    
    num_max = 0
    for cluster in clusters:
        loc = result[(result['label'] == cluster) & (result['predicted_label'] == label)]
        if len(loc) > num_max:
            num_max = len(loc)
        print('\t' + str(len(loc)), end='')
    
    print('\t' + str(num_max / label_count[1][label]))

(array([0, 1, 2, 3, 4], dtype=int32), array([ 33,  35,  16,   2, 187]))
		percent
0  |	33	1.0


In [18]:
for index, value in result[result['predicted_label'] == 4]['comment'].iteritems():
    print(index, '[', value, ']')

3 [ อย่าบังคับน้องเข้าประชุมเชียร์ อย่าลงโทษโดยเหตุผลงี่เง่าๆ เกิดก่อนไม่กี่ปีเอง ]
5 [ โลตัสใหญ่ไม่มีอะไรนะ โลตัสเอกเพรสแม่งไม่ไหวจริงๆ เกลียดขี้หน้าพนักงาน ]
7 [ มารยาทพนักงาน เซเว่นยังคุณภาพดีกว่าหน่อยนึง แบบว่าเหี้ยทั้งคู่เลย ]
8 [ ราคาสินค้า ตอนโปรโมชั่น บางอันหาไม่เจอ เหมือนพนักงานเอาไปซ่อน ให้ลูกค้าวิ่งไล่หา ไอ้ซั้ซไม่ใช่มอญซ่อนผ้า ปลรปภ บางสาขา หน้าตาดุมาก เวลาเข้าห้างนึกว่าทำไรผิดตลอดเวลา ]
9 [ เลิกเปิดเพลง ข้าวแสนดี กับอีเครื่องกรองน้ำเพียว ได้แล้ว ]
10 [ เช็คเอาท์  หน้าจะงอไปไหนสวัสดีค่ะมีอะไรให้ช่วยมั้ยคะ ]
15 [ เทสโก้โลตัสสาขาบางกะปิทีวีตัวนี้อยู่บริเวณหน้าร้าน   ชั้นลอยชั้นเดียวกับไปรษณีย์ใกล้ประตูออกลานจอดรถ ]
17 [ ตะคิดเป็น 25 สต 50สต 75สต ทำไม ในเมื่อลวท้ายด้วยราคาพวกนี้ ตอนคิดเงินปัดเศษขึ้นไปเต็มบาทเลยพอเราเอาเหรียญสตางค์ที่เป็นเศษให้ ชักสีหน้า ไม่อยากรับ ]
18 [ โลตลาด พนงปากตลาด นินทาลคเผาขน ตะโกนโหวกเหวกข้ามหัวลค วันหวยออกสนั่นเป็นพิเศษ เห็นพฤติกรรมแล้วแย่ ปรับปรุงภาพลักษณ์เถอะ ]
22 [ ทุกสาขา พนักงานหายไปไหน ต้องเดินตามหา พอเจอแลเวเรียกมาคิดเงิน ก็อารมณ์เสียหน้าบูด 

In [19]:
for index, value in result[result['predicted_label'] == 4]['tokenized_comment'].iteritems():
    print(index, value)

3 ['อย่า', 'อย่า', 'กี่', 'ปี', 'บังคับ', 'น้อง', 'เชียร์', 'งี่', 'เง่า']
5 ['พนักงาน', 'โลตัส', 'โลตัส', 'แม่ง', 'เกลียด', 'เอกเพรส']
7 ['พนักงาน', 'ดี', 'มารยาท', 'นึง', 'เซเว่น', 'คุณภาพ', 'เหี้ย', 'คู่']
8 ['พนักงาน', 'ลูกค้า', 'เหมือน', 'เวลา', 'เวลา', 'สินค้า', 'ราคา', 'เจอ', 'ตอน']
9 ['ดี', 'เลิก', 'น้ำ', 'เครื่อง', 'ข้าว', 'อี']
10 ['หน้า', 'เช็ค', 'งอ', 'สวัสดี']
15 ['สาขา', 'หน้า', 'รถ', 'ตัว', 'ร้าน', 'จอด', 'ชั้น', 'ชั้น', 'กะปิ']
17 ['เงิน', 'ราคา', 'ตอน', 'สีหน้า', 'เต็ม', 'บาท', 'เศษ', 'สตางค์', 'เศษ']
18 ['ปรับปรุง', 'พนง', 'แย่', 'ตะโกน', 'ข้าม', 'หวย', 'พิเศษ', 'พฤติกรรม']
22 ['พนักงาน', 'สาขา', 'ลูกค้า', 'หน้า', 'เงิน', 'เจอ', 'เดิน', 'หา', 'ขาย']
24 ['ซื้อ', 'สินค้า', 'ราคา', 'ติด', 'ป้าย']
31 ['พนักงาน', 'สาขา', 'เรื่อง', 'อบรม', 'เอ็กซ์เพรส', 'เซอร์วิสมายด์']
32 ['พนักงาน']
36 ['เวลา', 'เวลา', 'บริการ', 'สินค้า', 'ถาม', 'มารยาท', 'เจอ', 'ตอน', 'บ้าน']
37 ['โลตัส', 'พนง', 'ฐานะ']
38 ['พนักงาน', 'ลูกค้า', 'หน้า', 'โดน', 'ชอบ', 'บูด', 'เช้า', 'เห็นใจ', 'ฮะ']
41 ['พน